Data Preprocessing

Extracting Geometry from Shapefiles
1) [Section](#section)
2) [Subdivision](#subdivison)

Property Sales Data
1) [ps_full](#ps_full)
2) [ps_sec](#ps_sec)
3) [ps_sub](#ps_sub)

In [7]:
import pandas as pd
import geopandas as gpd
import geofeather as gf
import os

In [89]:
#Let's set the pandas option so we can see all the columns
pd.set_option('display.max_columns', None)

# Extracting Geometry from Shapefiles

The shapefiles provided by Miami's Open Data Hub contain geometries for displaying the boundaries of our subdivision and section. 

These files can be downloaded from Miami Open Data Hub:
- section: https://gis-mdc.opendata.arcgis.com/datasets/gis-section/explore
- subdivison: https://gis-mdc.opendata.arcgis.com/datasets/subdivision-boundary/explore

### section

In [29]:
section = gpd.read_file("data/section_shp/section.shp")

In [30]:
section.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2198 entries, 0 to 2197
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    2198 non-null   int64   
 1   TTRRSS      2198 non-null   object  
 2   TRSS        2198 non-null   object  
 3   SS          2198 non-null   object  
 4   ANGLE       2198 non-null   float64 
 5   CREATEDBY   0 non-null      float64 
 6   CREATEDDAT  0 non-null      float64 
 7   MODIFIEDBY  0 non-null      float64 
 8   MODIFIEDDA  0 non-null      float64 
 9   GlobalID    2198 non-null   object  
 10  Shape__Are  2198 non-null   float64 
 11  Shape__Len  2198 non-null   float64 
 12  DateOfSale  10 non-null     object  
 13  TTRRSS_1    38 non-null     float64 
 14  Subdivisio  38 non-null     float64 
 15  Price       38 non-null     float64 
 16  Folio       38 non-null     float64 
 17  Price_Per_  38 non-null     float64 
 18  BUILDING_A  38 non-null     float64 
 19

In [31]:
section['TTRRSS'] = section['TTRRSS'].astype("float")

### subdivision

In [32]:
subdivision = gpd.read_file('data/subdivision_shp/subdivision.shp')

In [33]:
subdivision.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 22592 entries, 0 to 22591
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    22592 non-null  int64   
 1   TTRRSS      22589 non-null  object  
 2   PB          22572 non-null  object  
 3   PG          22574 non-null  object  
 4   NAME        20793 non-null  object  
 5   TNUM        22563 non-null  float64 
 6   SUBCODE     22527 non-null  float64 
 7   SOURCE      4421 non-null   object  
 8   MOD_DATE    12978 non-null  object  
 9   MOD_TYPE    7139 non-null   object  
 10  EDITOR_NAM  5309 non-null   object  
 11  EDIT_DATE   5309 non-null   object  
 12  MODIFY_BY   3123 non-null   object  
 13  MODIFY_DAT  3123 non-null   object  
 14  GLOBALID    22592 non-null  object  
 15  Shape__Are  22592 non-null  float64 
 16  Shape__Len  22592 non-null  float64 
 17  geometry    22592 non-null  geometry
dtypes: float64(4), geometry(1), int64(1), 

In [63]:
subdivision['TTRRSS'] = subdivision['TTRRSS'].astype('float')

# Property Sales Data

The property sales data was retrieved and developed from the Miami Property Appraisers Bulk Data File Download at a cost of $50, and can be purchased here
- https://bbs.miamidade.gov/

## ps_full

In [74]:
ps_full = pd.read_pickle('data/pkl/ps_full.pkl')

In [75]:
subdivision = subdivision.loc[(subdivision['SUBCODE'].notna())]
subdivision['TTRRSS'] = subdivision['TTRRSS'].astype(float)
subdivision['SUBCODE'] = subdivision['SUBCODE'].astype(int)

/Users/nicolastobon/Library/Mobile Documents/com~apple~CloudDocs/Data Projects/Emerging-Hot-Spot-Analysis/env/lib/python3.9/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [76]:
ps_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221234 entries, 0 to 1221233
Data columns (total 12 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   DateOfSale            1221234 non-null  datetime64[ns]
 1   TTRRSS                1221234 non-null  float64       
 2   Subdivision           1221234 non-null  int64         
 3   Municipality          1221234 non-null  int64         
 4   Price                 1221234 non-null  float64       
 5   FOLIO                 1221234 non-null  int64         
 6   Price_Per_BAA         1205239 non-null  float64       
 7   BUILDING_ACTUAL_AREA  1221234 non-null  float64       
 8   BUILDING_HEATED_AREA  1221234 non-null  float64       
 9   LOT_SIZE              1221234 non-null  float64       
 10  ASSESSED_VAL_CUR      1221234 non-null  float64       
 11  TTRRSS-S              1221234 non-null  int64         
dtypes: datetime64[ns](1), float64(7), int64(4)

## ps_sec      

ps_sec aggregates the prices on the basis of geospatial section's which allow for a broader-view analysis of sale activity

In [82]:
ps_sec = ps_full.merge(section, left_on=['TTRRSS'], right_on=['TTRRSS'])

In [83]:
cols_to_drop = ['CREATEDBY', 'CREATEDDAT', 'MODIFIEDBY', 'MODIFIEDDA']
ps_sec.drop(cols_to_drop, inplace = True, axis = 1)

In [86]:
ps_sec

,DateOfSale_x,TTRRSS,Subdivision,Municipality,Price_x,FOLIO,Price_Per_BAA,BUILDING_ACTUAL_AREA,BUILDING_HEATED_AREA,LOT_SIZE_x,ASSESSED_VAL_CUR,TTRRSS-S,OBJECTID,TRSS,SS,ANGLE,GlobalID,Shape__Are,Shape__Len,DateOfSale_y,TTRRSS_1,Subdivisio,Price_y,Folio,Price_Per_,BUILDING_A,BUILDING_H,LOT_SIZE_y,ASSESSED_V,DateOfSa_1,TTRRSS_12,Subdivis_1,Price_1,Folio_1,Price_Per1,BUILDING_1,BUILDING_2,LOT_SIZE_1,ASSESSED_1,geometry
0,1900-01-31,534202.0,8,2,200000.0,1,inf,0.0,0.0,5650.0,444372.0,5342028,319,3202,02,-1.94165,2166e5ec-ca46-4f5d-943e-c54c590fe629,3.482966e+06,7475.212887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-80.12632 25.85662, -80.12832 25.856..."
1,1968-11-30,534202.0,8,2,18000.0,1,9.004502,1999.0,1999.0,5650.0,261075.0,5342028,319,3202,02,-1.94165,2166e5ec-ca46-4f5d-943e-c54c590fe629,3.482966e+06,7475.212887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-80.12632 25.85662, -80.12832 25.856..."
2,1970-03-31,534202.0,35,2,4000.0,1,10.000000,400.0,400.0,0.0,84075.0,53420235,319,3202,02,-1.94165,2166e5ec-ca46-4f5d-943e-c54c590fe629,3.482966e+06,7475.212887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-80.12632 25.85662, -80.12832 25.856..."
3,1970-12-31,534202.0,8,2,45000.0,1,13.493253,3335.0,3335.0,5650.0,619185.0,5342028,319,3202,02,-1.94165,2166e5ec-ca46-4f5d-943e-c54c590fe629,3.482966e+06,7475.212887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-80.12632 25.85662, -80.12832 25.856..."
4,1971-02-28,534202.0,7,2,109150.0,2,27.983273,3891.0,7052.0,5575.0,1506016.0,5342027,319,3202,02,-1.94165,2166e5ec-ca46-4f5d-943e-c54c590fe629,3.482966e+06,7475.212887,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-80.12632 25.85662, -80.12832 25.856..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1221229,2021-07-31,553925.0,17,30,4706000.0,2,612.882600,7596.5,7596.5,72777.0,2392997.0,55392517,1978,5925,25,-2.13602,bddf8b48-4822-42d5-9cd2-d84899f8fde4,3.244417e+06,7204.782227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-80.39574 25.61203, -80.39775 25.611..."
1221230,2021-09-30,553925.0,17,30,8450000.0,1,777.869833,10863.0,10863.0,70117.0,6482000.0,55392517,1978,5925,25,-2.13602,bddf8b48-4822-42d5-9cd2-d84899f8fde4,3.244417e+06,7204.782227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-80.39574 25.61203, -80.39775 25.611..."
1221231,2021-12-31,553925.0,17,30,19500000.0,1,113.032339,172517.0,172517.0,673835.0,16288000.0,55392517,1978,5925,25,-2.13602,bddf8b48-4822-42d5-9cd2-d84899f8fde4,3.244417e+06,7204.782227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-80.39574 25.61203, -80.39775 25.611..."
1221232,2022-01-31,553925.0,17,30,10546100.0,1,inf,0.0,0.0,539298.0,4314384.0,55392517,1978,5925,25,-2.13602,bddf8b48-4822-42d5-9cd2-d84899f8fde4,3.244417e+06,7204.782227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-80.39574 25.61203, -80.39775 25.611..."


In [87]:
ps_sec['DateOfSale_x'] = ps_sec['DateOfSale_x'].dt.strftime('%Y-%m-%d')

In [88]:
ps_sec = gpd.GeoDataFrame(ps_sec)

In [89]:
ps_sec.to_file('ps_sec.shp')

C:\Users\nicolastobon\AppData\Local\Temp\ipykernel_11428\3924269689.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ps_sec.to_file('ps_sec.shp')


## ps_sub

ps_sub aggregates the prices on the basis of geospatial subdivision's which allows for a granular-view analysis of sale activity

In [90]:
ps_sub= ps_full.merge(subdivision, left_on=['TTRRSS', 'Subdivision'], right_on=['TTRRSS', 'SUBCODE'])

In [79]:
cols_to_drop = ['PB', 'PG', 'TNUM', 'SOURCE', 'MOD_DATE', 'MOD_TYPE', 'EDITOR_NAM', 'EDIT_DATE', 'MODIFY_BY', 'MODIFY_DAT']
ps_sub.drop(cols_to_drop, inplace = True, axis = 1)

In [91]:
ps_sub

,DateOfSale,TTRRSS,Subdivision,Municipality,Price,FOLIO,Price_Per_BAA,BUILDING_ACTUAL_AREA,BUILDING_HEATED_AREA,LOT_SIZE,ASSESSED_VAL_CUR,TTRRSS-S,OBJECTID,PB,PG,NAME,TNUM,SUBCODE,SOURCE,MOD_DATE,MOD_TYPE,EDITOR_NAM,EDIT_DATE,MODIFY_BY,MODIFY_DAT,GLOBALID,Shape__Are,Shape__Len,geometry
0,1900-01-31,534202.0,8,2,200000.0,1,inf,0.0,0.0,5650.0,444372.0,5342028,19816,44,67,BISCAYNE BEACH SUB,0.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,563c911a-6fd3-4eb4-934f-32b27c16d1d4,295389.394531,2585.373308,"POLYGON ((-80.12482 25.86992, -80.12481 25.869..."
1,1968-11-30,534202.0,8,2,18000.0,1,9.004502,1999.0,1999.0,5650.0,261075.0,5342028,19816,44,67,BISCAYNE BEACH SUB,0.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,563c911a-6fd3-4eb4-934f-32b27c16d1d4,295389.394531,2585.373308,"POLYGON ((-80.12482 25.86992, -80.12481 25.869..."
2,1970-12-31,534202.0,8,2,45000.0,1,13.493253,3335.0,3335.0,5650.0,619185.0,5342028,19816,44,67,BISCAYNE BEACH SUB,0.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,563c911a-6fd3-4eb4-934f-32b27c16d1d4,295389.394531,2585.373308,"POLYGON ((-80.12482 25.86992, -80.12481 25.869..."
3,1971-02-28,534202.0,8,2,32000.0,1,17.021277,1880.0,1830.0,5650.0,454742.0,5342028,19816,44,67,BISCAYNE BEACH SUB,0.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,563c911a-6fd3-4eb4-934f-32b27c16d1d4,295389.394531,2585.373308,"POLYGON ((-80.12482 25.86992, -80.12481 25.869..."
4,1971-04-30,534202.0,8,2,23200.0,1,6.451613,3596.0,3596.0,5650.0,643463.0,5342028,19816,44,67,BISCAYNE BEACH SUB,0.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,563c911a-6fd3-4eb4-934f-32b27c16d1d4,295389.394531,2585.373308,"POLYGON ((-80.12482 25.86992, -80.12481 25.869..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1917043,2022-09-30,534113.0,90,1,2190400.0,1,21.105779,103782.0,103782.0,71003.0,7900000.0,53411390,9466,137,51,TACOLCY GARDENS,0.0,90,NaN,10291999,NaN,NaN,NaN,NaN,NaN,8aa8fec4-5b94-428d-bd27-8613d5a6c1d6,27284.050781,779.665753,"POLYGON ((-80.20669 25.83044, -80.20654 25.830..."
1917044,2022-09-30,534128.0,28,30,8300000.0,1,400.966184,20700.0,19660.0,80586.0,1663488.0,53412828,4520,111,81,AIR MARINE TERMINALS,0.0,28,NaN,10291999,NaN,NaN,NaN,NaN,NaN,9a277128-86e5-4aad-8550-66470751174b,9183.541016,406.848500,"POLYGON ((-80.25352 25.80151, -80.25313 25.801..."
1917045,2022-09-30,534128.0,28,30,8300000.0,1,400.966184,20700.0,19660.0,80586.0,1663488.0,53412828,4898,111,81,AIR MARINE TERMINALS,0.0,28,NaN,10291999,NaN,NaN,NaN,NaN,NaN,4eefc36a-86c9-40c2-b5f9-8506052e5212,9784.988281,424.739075,"POLYGON ((-80.25490 25.80254, -80.25452 25.802..."
1917046,2022-09-30,534128.0,28,30,8300000.0,1,400.966184,20700.0,19660.0,80586.0,1663488.0,53412828,12193,111,81,AIR MARINE TERMINALS,0.0,28,NaN,10291999,NaN,NaN,NaN,NaN,NaN,b2d66df1-00f3-4e68-b833-0fec4ae6bbdc,44272.802734,1162.305407,"POLYGON ((-80.25508 25.80239, -80.25441 25.801..."


In [80]:
ps_sub['DateOfSale'] = ps_sub['DateOfSale'].dt.strftime('%Y-%m-%d')

In [81]:
ps_sub = gpd.GeoDataFrame(ps_sub)

In [69]:
ps_sub.to_file('ps_sub.shp', geometry = 'geometry')

C:\Users\nicolastobon\AppData\Local\Temp\ipykernel_11428\3647195241.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ps_sub.to_file('ps_sub.shp', geometry = 'geometry')
